In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Spatial Attention Mechanism
def spatial_attention(x):
    avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)
    max_pool = tf.reduce_max(x, axis=-1, keepdims=True)
    concat = tf.concat([avg_pool, max_pool], axis=-1)
    conv = Conv2D(1, (7, 7), padding="same", activation="sigmoid")(concat)
    return x * conv  # Apply attention

# Load Pre-trained VGG16 Model
def build_model(input_shape=(256, 256, 3)):
    inputs = Input(shape=input_shape)

    # Pre-trained VGG16 without the fully connected layers
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)(inputs)

    # Adding spatial attention mechanism
    attention = spatial_attention(vgg16)

    # Apply Global Average Pooling
    gap = GlobalAveragePooling2D()(attention)

    # Fully connected layers
    dense1 = Dense(512, activation='relu')(gap)
    dropout1 = Dropout(0.5)(dense1)
    dense2 = Dense(256, activation='relu')(dropout1)
    output = Dense(1, activation='sigmoid')(dense2)  # Binary classification (0 or 1 for landslide)

    # Create the final model
    model = Model(inputs=inputs, outputs=output)

    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Example usage
model = build_model()
model.summary()
